# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the SQL questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create code to: (put these each into a function)
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- Return the text of the reviews for the most reviewed restaurant. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [14]:
## Connect to DB server on AWS
import mysql.connector 
import config
import requests
import time  # This is just to be able to make multiple calls to the Yelp API in spaced out time intervals

cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password
)
cursor = cnx.cursor()

In [11]:
# cursor.close()
# cnx.close()

In [15]:
## Create new DB 
from mysql.connector import errorcode
db_name = 'yelp'
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [ ]:
# Create a table for the Businesses
create_query = """
CREATE TABLE businesses (
    business_id varchar(20) NOT NULL UNIQUE,
    name varchar(50) NOT NULL,
    review_count int(10) NOT NULL,
    price varchar(7) NOT NULL,
    avg_rating float NOT NULL
    );

"""
cursor.execute(create_query)


In [ ]:
# Did this to modify the business_id column

# cursor.execute("""
#                ALTER TABLE businesses
#                MODIFY business_id varchar(50);
#                """)

# make business ID the primary key

# cursor.execute("""
#                ALTER TABLE businesses
#                ADD CONSTRAINT business_id PRIMARY KEY (business_id);
#                """)

In [ ]:
# Create a table for the reviews
create_query = """
CREATE TABLE reviews (
    review_id int(7) NOT NULL UNIQUE AUTO_INCREMENT,
    user_name varchar(50) NOT NULL,
    review_text varchar(500) NOT NULL,
    time_created datetime NOT NULL,
    rating float NOT NULL
    );

"""
cursor.execute(create_query)

# ['reviews']['user']['name']
# ['reviews']['text']
# ['reviews']['time_created']
# ['reviews']['rating']

In [ ]:
# cursor.execute("""
#                ALTER TABLE reviews
#                MODIFY review_id varchar(50) NOT NULL UNIQUE;
#                """)
# cursor.execute("""
#                ALTER TABLE reviews
#                ADD COLUMN business_id varchar(50) NOT NULL;
#                """)

# cursor.execute("""
#                ALTER TABLE reviews
#                ADD CONSTRAINT business_id
#                FOREIGN KEY (business_id) REFERENCES yelp.businesses(business_id);
#                """)

## Part 2: Create ETL pipeline for the business data from the API

In [16]:
# Write a function to make a call to the yelp API

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()['businesses']
    return data

# yelp_call(url_params, api_key)[1]

In [17]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB

def parse_results(results):
    fields = ['id', 'name', 'review_count', 'rating', 'price']
    parsed_results = []
    for result in results:
        parsed_result = {field: (result[field] if field in result.keys() else 'Null') 
                         for field in fields}
        parsed_results.append(parsed_result)
    return parsed_results

parsed_results = parse_results(yelp_call(url_params, api_key))

In [18]:
# Write a function to take your parsed data and insert it into the DB

def insert_business(parsed_results):
    business_tuples = [(parsed_result['id'],
                        parsed_result['name'],
                        parsed_result['review_count'],
                        parsed_result['rating'],
                        parsed_result['price'])
                        for parsed_result in parsed_results]
    
    insert_statement = "INSERT INTO businesses (business_id, name, review_count, avg_rating, price) VALUES (%s, %s, %s, %s, %s)"
    cursor.executemany(insert_statement,business_tuples)
    cnx.commit()



In [25]:
parsed_result = parse_results(yelp_call(url_params, api_key))[0]
business_tuples = [(parsed_result['id'],
                        parsed_result['name'],
                        parsed_result['review_count'],
                        parsed_result['rating'],
                        parsed_result['price']
print(business_tuples)

SyntaxError: invalid syntax (<ipython-input-25-17f11754a98e>, line 7)

In [19]:
# Write a script that combines the three functions above into a single process.

term = 'cocktails'
location = '7th St., NYC'
SEARCH_LIMIT = 50
api_key = 'gHzgfOrEmRXbfcaOfojNz43Jsdarm-kRyCuA53XRYaZ0WS06fwrokxGky8S2gSNDPieCXsaYwN4w0Oh7g3hvfkkz7-na1l27iiGNEhvaqBmm_u_Lkxcfy1_IO9yxXXYx'
url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT
            }

def all_results(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results = yelp_call(url_params, api_key)
        parsed_results = parse_results(results)
        insert_business(parsed_results)
        time.sleep(1) #Wait a second
        cur += 50


In [20]:
all_results(url_params, api_key)

1300 total matches found.


## Part 3: Create ETL pipeline for the restaurant review data from the API

In [21]:
# write a function that takes a business id 
# and makes a call to the API for reivews

def review_call(api_key, business_id):
    url = 'https://api.yelp.com/v3/businesses/{}/reviews'.format(business_id)
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers)
    return response.json()['reviews']

In [27]:
# review_call(api_key,business_ids[0][0])
# print(business_ids[0][0])

In [22]:
# Write a function to parse out the relevant information from the reviews

def parse_reviews(reviews, business_id):
    fields = ['id', 'user', 'time_created', 'rating', 'text']
    parsed_reviews = []
    for review in reviews:
        parsed_review = {}
        for field in fields:
            parsed_review['business_id'] = business_id
            if field in review.keys() and field != 'user':
                parsed_review[field] = review[field]
            elif field == 'user':
                parsed_review['user_name'] = review['user']['name']
            else:
                parsed_review[field] = 'Null'
        parsed_reviews.append(parsed_review)
    return parsed_reviews



In [23]:
# write a function to insert the parsed data into the reviews table
def insert_reviews(parsed_reviews):
    reviews_tuples = [(parsed_review['id'],
                        parsed_review['user_name'],
                        parsed_review['text'],
                        parsed_review['time_created'],
                        parsed_review['rating'],
                        parsed_review['business_id'])
                        for parsed_review in parsed_reviews]
    insert_statement = """
                       INSERT INTO reviews (review_id, user_name, review_text, 
                                               time_created, rating, business_id) 
                       VALUES (%s, %s, %s, %s, %s, %s)
                       ;"""
    cursor.executemany(insert_statement,reviews_tuples)
    cnx.commit()

In [24]:
# combine the functions above into a single script  

api_key = 'gHzgfOrEmRXbfcaOfojNz43Jsdarm-kRyCuA53XRYaZ0WS06fwrokxGky8S2gSNDPieCXsaYwN4w0Oh7g3hvfkkz7-na1l27iiGNEhvaqBmm_u_Lkxcfy1_IO9yxXXYx'

cursor.execute("""
            SELECT
            business_id
            FROM
            businesses
            ;""")
business_ids = [element[0] for element in cursor.fetchall()]

for business_id in business_ids:
    reviews = review_call(api_key,business_id)
    parsed_reviews = parse_reviews(reviews, business_id)
    insert_reviews(parsed_reviews)

## Part 4: Write SQL queries that will answer the questions posed. 

In [ ]:
# create connection

In [ ]:
# execute SQL queries

# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

In [ ]:
def parse_results(results):
    
    return parsed_results

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time



def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results = yelp_call(url_params, api_key)
        parsed_results = parse_results(results)
        db_insert(parsed)
        time.sleep(1) #Wait a second
        cur += 50

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id INT AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```

### Using DB:
    
For this lab, you can either store the data on one DB or put in on both of the partners DBs. If you decide to put it on one DB, you want to make sure both partners have access to it.  To do this you want to add a user to your DB.  

[how to add a new user](https://howchoo.com/g/mtm3zdq2nzv/how-to-add-a-mysql-user-and-grant-privileges)